In [ ]:
#Extracting the zipfile in google drive
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/audio.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()


Mounted at /content/drive/


In [ ]:
import sys
sys.path.insert(0, '/tmp/')
sys.path.insert(0,'/content/drive/MyDrive/')
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
#from audio_classifier import AudioClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet121


In [ ]:
import tensorflow
print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.12.0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from denseNet_audio_trans import denseNetAudioTransform
#from audio_classifier import AudioClassifier
from audio_utils import AudioUtil
from numpy import savez_compressed
from numpy import load

In [ ]:
#read path csv
df=pd.read_csv("/tmp/relativepaths.csv")
X=df['relative_path']
y=df["classID"]
#split train\test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y,test_size=0.2, random_state=1)
#split train\val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True,test_size=0.2, random_state=1)

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
y_val=to_categorical(y_val)

In [ ]:
X_train.index

Int64Index([2274,  936, 8497, 3712, 6815, 7744, 2108, 6143, 5326, 4658,
            ...
            3866, 3135,  182,  447, 6814, 5898,  696,  762, 5207, 6120],
           dtype='int64', length=5588)

In [ ]:
y_train.shape

(5588, 10)

In [ ]:
atrans=denseNetAudioTransform(df=df, data_path="/tmp/audio/")

In [ ]:
example_dataset=tensorflow.data.Dataset.from_tensor_slices((X_train.index))
example_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [ ]:
for idx in example_dataset:
  print(idx)
  break

tf.Tensor(2274, shape=(), dtype=int64)


In [ ]:
def audio_transform(idx):
  idx=idx.numpy()
  return atrans[idx]

def sgram_reshape(sgram, label):
  sgram=sgram.numpy()
  sgram_new=np.moveaxis(sgram,0,-1)
  return sgram_new, label

def _fixup_shape(x, y):
    x.set_shape([None, 128, 400, 3]) # n, h, w, c
    y.set_shape([None, 10]) # n, nb_classes
    return x, y

#gen_dataset=example_dataset.from_generator(generator_func, output_types=tensorflow.int64)
#print(gen_dataset)

map_dataset=example_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                 inp=[i],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
print(map_dataset)

for sgrams, class_id in map_dataset:
  print(sgrams.shape)
  print(class_id)
  break

map2_dataset=map_dataset.map(lambda sgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                 inp=[sgram, label],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
print(map2_dataset)

for sgram, class_id in map2_dataset:
  print(sgram.shape)
  print(class_id)
  break

<_ParallelMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), TensorSpec(shape=<unknown>, dtype=tf.float64, name=None))>
(3, 128, 400)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)
<_ParallelMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), TensorSpec(shape=<unknown>, dtype=tf.float64, name=None))>
(128, 400, 3)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], shape=(10,), dtype=float64)


In [ ]:
#train dataset
train_dataset=tensorflow.data.Dataset.from_tensor_slices((X_train.index))
train_dataset=train_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                 inp=[i],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
train_dataset=train_dataset.map(lambda sgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                 inp=[sgram, label],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
print(train_dataset)
train_dataset=train_dataset.cache()
train_dataset=train_dataset.shuffle(len(train_dataset))
train_dataset=train_dataset.batch(32)
print(train_dataset)
train_dataset=train_dataset.map(_fixup_shape)
print(train_dataset)
train_dataset=train_dataset.prefetch(tensorflow.data.AUTOTUNE)
print(train_dataset)
print(len(train_dataset))

<_ParallelMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), TensorSpec(shape=<unknown>, dtype=tf.float64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), TensorSpec(shape=<unknown>, dtype=tf.float64, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(None, 128, 400, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 10), dtype=tf.float64, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 400, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 10), dtype=tf.float64, name=None))>
175


In [ ]:
val_dataset=tensorflow.data.Dataset.from_tensor_slices((X_val.index))
val_dataset=val_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                 inp=[i],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
val_dataset=val_dataset.map(lambda sgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                 inp=[sgram, label],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)

val_dataset=val_dataset.cache()
val_dataset=val_dataset.batch(32)
val_dataset=val_dataset.map(_fixup_shape)
val_dataset=val_dataset.prefetch(tensorflow.data.AUTOTUNE)
print(val_dataset)
print(len(val_dataset))

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 400, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 10), dtype=tf.float64, name=None))>
44


In [ ]:
test_dataset=tensorflow.data.Dataset.from_tensor_slices((X_test.index))
test_dataset=test_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                 inp=[i],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)
test_dataset=test_dataset.map(lambda isgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                 inp=[sgram, label],
                                                                 Tout=[tensorflow.float64, tensorflow.float64]),
                                num_parallel_calls=tensorflow.data.AUTOTUNE)

test_dataset=test_dataset.cache()
test_dataset=test_dataset.batch(32)
test_dataset=test_dataset.map(_fixup_shape)
test_dataset=test_dataset.prefetch(tensorflow.data.AUTOTUNE)
print(test_dataset)
print(len(test_dataset))

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 400, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 10), dtype=tf.float64, name=None))>
55


In [ ]:
for spec,label in val_dataset:
  print(spec.shape)
  print(label)
  break

(32, 128, 400, 3)
tf.Tensor(
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 

In [ ]:
dense121_model=DenseNet121(weights='imagenet',include_top=False, input_shape=(128,400,3))

NameError: name 'DenseNet121' is not defined

In [ ]:
x=dense121_model.output
x= layers.GlobalAveragePooling2D()(x)
x= layers.BatchNormalization()(x)
x= layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x= layers.Dense(1024,activation='relu')(x)
x= layers.Dense(512,activation='relu')(x)
x= layers.BatchNormalization()(x)
x= layers.Dropout(0.5)(x)
preds=layers.Dense(10,activation='softmax')(x)

model=keras.Model(inputs=dense121_model.input,outputs=preds)

In [ ]:
custom_optimizer = tensorflow.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-3)
model.compile(optimizer=custom_optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 406, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 200, 64)  9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 200, 64)  256         ['conv1/conv[0][0]']         

In [ ]:
history=model.fit(train_dataset, epochs=70, batch_size=32, validation_data=val_dataset)

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "r", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
plt.clf()
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "r", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Applying the standard 10-fold CV for pre-defined fols of Sound8k dataset

In [ ]:
#Extracting the zipfile in google drive
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/audio.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()


Mounted at /content/drive/


In [ ]:
import sys
sys.path.insert(0, '/tmp/')
sys.path.insert(0,'/content/drive/MyDrive/')
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
#from audio_classifier import AudioClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet121
import tensorflow
print("Tensorflow-version:", tensorflow.__version__)
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from denseNet_audio_trans import denseNetAudioTransform
#from audio_classifier import AudioClassifier
from audio_utils import AudioUtil
from numpy import savez_compressed
from numpy import load

Tensorflow-version: 2.12.0


In [ ]:
#read path csv
df=pd.read_csv("/tmp/relativepaths.csv")
X=df['relative_path']
y=df["classID"]

In [ ]:
atrans=denseNetAudioTransform(df=df, data_path="/tmp/audio/")

In [ ]:
def audio_transform(idx):
  idx=idx.numpy()
  return atrans[idx]

def sgram_reshape(sgram, label):
  sgram=sgram.numpy()
  sgram_new=np.moveaxis(sgram,0,-1)
  return sgram_new, label

def _fixup_shape(x, y):
    x.set_shape([None, 128, 400, 3]) # n, h, w, c
    y.set_shape([None, 10]) # n, nb_classes
    return x, y

In [ ]:
#folds dict that has keys with fold names and list(index) as values
folds={}
for path, idx in zip(X,X.index):
    if path.split("/")[1] in folds:
        folds[path.split("/")[1]].append(idx)
    else:
        folds[path.split("/")[1]]=[idx]
folds.keys()

dict_keys(['fold5', 'fold10', 'fold2', 'fold6', 'fold1', 'fold9', 'fold7', 'fold4', 'fold3', 'fold8'])

In [ ]:
acc_per_fold = []
loss_per_fold = []
for fold in folds.keys():
  #creating train and val index sets
  val_idx=folds[fold]
  train_folds=[i for i in list(folds.keys()) if i!=fold]
  train_idx=[]
  for train_fold in train_folds:
    train_idx=train_idx+folds[train_fold]
  #preprocessing------------------------------------------------------------------------------------------------------
  #train dataset
  train_dataset=tensorflow.data.Dataset.from_tensor_slices((train_idx))
  train_dataset=train_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                  inp=[i],
                                                                  Tout=[tensorflow.float64, tensorflow.float64]),
                                  num_parallel_calls=tensorflow.data.AUTOTUNE)
  train_dataset=train_dataset.map(lambda sgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                  inp=[sgram, label],
                                                                  Tout=[tensorflow.float64, tensorflow.float64]),
                                  num_parallel_calls=tensorflow.data.AUTOTUNE)
  train_dataset=train_dataset.cache()
  train_dataset=train_dataset.shuffle(len(train_dataset))
  train_dataset=train_dataset.batch(32)
  train_dataset=train_dataset.map(_fixup_shape)
  train_dataset=train_dataset.prefetch(tensorflow.data.AUTOTUNE)
  #val dataset
  val_dataset=tensorflow.data.Dataset.from_tensor_slices((val_idx))
  val_dataset=val_dataset.map(lambda i: tensorflow.py_function(func=audio_transform,
                                                                  inp=[i],
                                                                  Tout=[tensorflow.float64, tensorflow.float64]),
                                  num_parallel_calls=tensorflow.data.AUTOTUNE)
  val_dataset=val_dataset.map(lambda sgram, label: tensorflow.py_function(func=sgram_reshape,
                                                                  inp=[sgram, label],
                                                                  Tout=[tensorflow.float64, tensorflow.float64]),
                                  num_parallel_calls=tensorflow.data.AUTOTUNE)
  val_dataset=val_dataset.cache()
  val_dataset=val_dataset.batch(32)
  val_dataset=val_dataset.map(_fixup_shape)
  val_dataset=val_dataset.prefetch(tensorflow.data.AUTOTUNE)
  #------------------------------------------------------------------------------------------------------------------
  #model architecture------------------------------------------------------------------------------------------------
  dense121_model=DenseNet121(weights='imagenet',include_top=False, input_shape=(128,400,3))
  x=dense121_model.output
  x= layers.GlobalAveragePooling2D()(x)
  x= layers.BatchNormalization()(x)
  x= layers.Dropout(0.5)(x)
  x = layers.Flatten()(x)
  x= layers.Dense(1024,activation='relu')(x)
  x= layers.Dense(512,activation='relu')(x)
  x= layers.BatchNormalization()(x)
  x= layers.Dropout(0.5)(x)
  preds=layers.Dense(10,activation='softmax')(x)
  model=keras.Model(inputs=dense121_model.input,outputs=preds)
  #-----------------------------------------------------------------------------------------------------------------
  #compile and fit
  custom_optimizer = tensorflow.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-3)
  model.compile(optimizer=custom_optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])
  history=model.fit(train_dataset, epochs=70, batch_size=32, verbose=0)
  #evaluate
  # Generate generalization metrics
  scores = model.evaluate(val_dataset, verbose=0)
  print(f'Score for {fold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

#Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


Score for fold5: loss of 0.6008174419403076; accuracy of 86.00427508354187%
Score for fold10: loss of 0.9429890513420105; accuracy of 83.03464651107788%
Score for fold2: loss of 1.2189068794250488; accuracy of 77.2522509098053%
Score for fold6: loss of 1.560352087020874; accuracy of 76.18468999862671%
Score for fold1: loss of 1.948787808418274; accuracy of 72.16494679450989%
Score for fold9: loss of 1.1548210382461548; accuracy of 83.45588445663452%
Score for fold7: loss of 1.1312953233718872; accuracy of 82.10023641586304%
Score for fold4: loss of 0.658184826374054; accuracy of 84.94949340820312%
Score for fold3: loss of 2.9381492137908936; accuracy of 66.59459471702576%
Score for fold8: loss of 2.4088268280029297; accuracy of 70.59553265571594%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.6008174419403076 - Accuracy: 86.00427508354187%
---------------